https://www.analyticsvidhya.com/blog/2021/12/text-classification-of-news-articles/

ปัญหาของข้อมูลชุดนี้


In [2]:
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [3]:
path = '../wordcloud_style/'
font = path+'Chomsky.otf'
pics = path+'pics.png'

#### import data from cleansing

In [4]:
df = pd.read_csv('../data/cleaned/cleaned-data.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4480 entries, 0 to 4479
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      4480 non-null   object
 1   label     4480 non-null   object
 2   headline  4480 non-null   object
 3   snippet   4472 non-null   object
 4   url       4480 non-null   object
dtypes: object(5)
memory usage: 175.1+ KB


#### Values Checking

In [6]:
def find_string(a, b, c):
    '''
    ฟังชั่นสำหรับหา String จาก column อย่างเฉพาะเจาะจง
    a = DataFrame Column
    b = 
    c = ตัวอักษรที่จะค้นหา
    Return = DataFrame
    '''
    string = []
    index = []
    for i in df[a]:
        string.append(i.find(c))
    for j in df.index:
        index.append(j)
    df[b] = string
    df['index'] = index
    return df[['index',a,b]].loc[df[b] > -1]

find_string('url', 'ind', 'politics')

,index,url,ind
9,9,https://www.nytimes.com/2019/12/31/us/politics...,38
33,33,https://www.nytimes.com/2020/01/01/us/politics...,38
38,38,https://www.nytimes.com/2020/01/01/us/politics...,38
68,68,https://www.nytimes.com/2020/01/01/us/politics...,38
85,85,https://www.nytimes.com/2020/01/01/us/politics...,38
...,...,...,...
4461,4461,https://www.nytimes.com/2020/01/31/us/politics...,38
4467,4467,https://www.nytimes.com/2020/01/31/us/politics...,38
4469,4469,https://www.nytimes.com/2020/01/31/us/politics...,38
4474,4474,https://www.nytimes.com/2020/01/31/us/politics...,38


In [7]:
def count_values(a):
    '''
    a = ชื่อ column
    ฟั่งชั่นนับจำนวน Label ทั้งหมด
    Return = DataFrame 
    '''
    label = []
    u_names = []
    count = []
    for i in df[a]:
        label.append(i)
    for names in label:
        if names not in u_names:
            names = str(names)
            u_names.append(names)
    for num in u_names:
        count.append(label.count(num))
    return pd.DataFrame({'Label': u_names, 'Numbers': count})

Num_values = count_values('label')
Num_values[['Label', 'Numbers']].loc[Num_values['Numbers'] > 0]

,Label,Numbers
0,us,287
1,opinion,503
2,health,49
3,crosswords,43
4,todayspaper,28
5,world,482
6,politics,494
7,science,61
8,arts,346
9,business,297


In [8]:
df.loc[df['label'] == 'health']

,date,label,headline,snippet,url,ind,index
2,2020-01-01,health,F.D.A. Plans to Ban Most E-Cigarette Flavors b...,The tobacco and vaping industries and conserva...,https://www.nytimes.com/2019/12/31/health/e-ci...,-1,2
65,2020-01-01,health,A.I. Is Learning to Read Mammograms,Computers that are trained to recognize patter...,https://www.nytimes.com/2020/01/01/health/brea...,-1,65
66,2020-01-01,health,New Injection Method Makes an Old TB Vaccine F...,Giving the vaccine intravenously to monkeys pr...,https://www.nytimes.com/2020/01/01/health/tube...,-1,66
130,2020-01-02,health,"It Looks Like Health Insurance, but It’s Not. ...",Some state regulators are scrutinizing nonprof...,https://www.nytimes.com/2020/01/02/health/chri...,-1,130
175,2020-01-02,health,"With Partial Flavor Ban, Trump Splits the Diff...",An initiative to reduce the use of e-cigarette...,https://www.nytimes.com/2020/01/02/health/flav...,-1,175
312,2020-01-03,health,Older People Need Geriatricians. Where Will Th...,The medical profession has been troubled for y...,https://www.nytimes.com/2020/01/03/health/geri...,-1,312
600,2020-01-06,health,A.I. Comes to the Operating Room,Images made by lasers and read by computers ca...,https://www.nytimes.com/2020/01/06/health/arti...,-1,600
894,2020-01-08,health,Cancer Death Rate in U.S. Sees Sharpest One-Ye...,Breakthrough treatments for lung cancer and me...,https://www.nytimes.com/2020/01/08/health/canc...,-1,894
903,2020-01-08,health,"The Flu Season May Yet Turn Ugly, C.D.C. Warns",Almost as many people are falling ill as did t...,https://www.nytimes.com/2020/01/08/health/flu-...,-1,903
956,2020-01-08,health,These Patients Are Hard to Treat,A study examined a popular approach that coord...,https://www.nytimes.com/2020/01/08/health/camd...,-1,956


#### Word Cloud

In [55]:
image = Image.open(pics)

In [56]:
class 

In [57]:
#word_cloud('snippet','label','health')

In [1]:
edit

IPython will make a temporary file named: C:\Users\Marwan\AppData\Local\Temp\ipython_edit_noqqa0kv\ipython_edit_q78ziso7.py


In [2]:
__name__

'__main__'

In [3]:
i = __name__

In [4]:
type(i)

str

In [5]:
type(__name__)

str